In [41]:
import pandas as pd
import numpy as np


od_df = pd.read_csv("MigrationPatternsData/od.csv")



## Group od.cs by state

In [42]:
df_grouped_multiple = od_df.groupby(['o_state_name','d_state_name']).agg({'n': ['sum']})
df_grouped_multiple = df_grouped_multiple.reset_index()

df_grouped_multiple.columns = ['o_state_name','d_state_name','n']
df_d = df_grouped_multiple.groupby(['d_state_name']).agg({'n':'sum'})
df_d = df_d.reset_index()
df_d.columns = ['state','inbound_number']
df_grouped_multiple.to_csv("state_migration.csv")

# create pivot table
od_pivot = od_df.pivot_table(index="o_state_name", columns="d_state_name", values="n", aggfunc="sum")
od_pivot.to_csv("state_migration_pivot.csv")


In [43]:
# create state summary table

states = df_grouped_multiple['o_state_name'].unique() # list to store all state names

df_summary = {'state':[],
'inbound_migration':[],
'outbound_migration':[],
'within_state_migration':[]}

df_summary = pd.DataFrame(df_summary)

df_summary['state'] = states

df_grouped_new = df_grouped_multiple
# fill the column of "within_state_migration"
for i in range(len(df_grouped_new)):
    if df_grouped_new.loc[i]['o_state_name']==df_grouped_new.loc[i]['d_state_name']:
        s = df_grouped_new.loc[i]['o_state_name']
        df_summary.loc[df_summary['state']==s,['within_state_migration']] = df_grouped_new.loc[i]['n']
        df_grouped_new.drop(i, axis='index', inplace=True) # drop the row of within_state_migration






In [44]:
df_d = df_grouped_multiple.groupby(['d_state_name']).agg({'n':'sum'})
df_d = df_d.reset_index()
df_d.columns = ['state','inbound_number']

# df_d['state'] == df_summary['state']
if df_d['state'].equals(df_summary['state']):
    df_summary['inbound_migration'] = df_d['inbound_number']

df_o = df_grouped_multiple.groupby(['o_state_name']).agg({'n':'sum'})
df_o = df_o.reset_index()
df_o.columns = ['state','outbound_number']

# df_d['state'] == df_summary['state']
if df_o['state'].equals(df_summary['state']):
    df_summary['outbound_migration'] = df_o['outbound_number']


In [46]:
#df_summary.info()
df_summary['within_state_migration'] = df_summary['within_state_migration'].astype(int)
df_summary.info()
df_summary.to_csv("state_migration_summary.csv")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   state                   51 non-null     object
 1   inbound_migration       51 non-null     int64 
 2   outbound_migration      51 non-null     int64 
 3   within_state_migration  51 non-null     int64 
dtypes: int64(3), object(1)
memory usage: 1.7+ KB


## Combine household_income data with migration data

In [47]:
df_income = pd.read_csv("state_household_income.csv")

df_income = df_income.replace({'state_name': {'DC': 'District of Columbia'}})

for i in range(len(df_income.index)):
    df_income.iloc[i,1] = df_income.iloc[i,1].upper()

df_income = df_income.sort_values(by='state_name')
df_income = df_income.reset_index(drop=True)
df_d = df_d.sort_values(by='state')

df_income['inbound_migration'] = df_d['inbound_number']
df_income =df_income.rename(columns={"mean": "average_household_income_median"})

df_income.to_csv("state_income_for_viz.csv")

In [48]:
df_income_sim = df_income.drop(df_income.iloc[:,:21],axis=1)
df_income_sim.corr()

,average_household_income_median,inbound_migration
average_household_income_median,1.000000,0.068571
inbound_migration,0.068571,1.000000
